In [11]:
#필요 라이브러리 로드
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

sns.set(style='whitegrid')

In [4]:
#KOPIS 표본데이터 로드
df = pd.read_excel("F:/drive/KOPIS/sample_01/sample_2020_1q_01.xlsx", encoding='utf-8')

In [5]:
#데이터 간략히 살펴보기
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 63 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   전송사업자코드           30000 non-null  object 
 1   전송사업자명            30000 non-null  object 
 2   공연시설코드            30000 non-null  object 
 3   시설특성              30000 non-null  object 
 4   개관연도              29636 non-null  float64
 5   주소                30000 non-null  object 
 6   편의시설_레스토랑 여부      30000 non-null  object 
 7   편의시설_카페 여부        30000 non-null  object 
 8   편의시설_편의점 여부       30000 non-null  object 
 9   편의시설_놀이방 여부       30000 non-null  object 
 10  편의시설_수유실 여부       30000 non-null  object 
 11  장애인시설_주차장 여부      30000 non-null  object 
 12  장애인시설_화장실 여부      30000 non-null  object 
 13  장애인시설_경사로 여부      30000 non-null  object 
 14  장애인시설_전용엘리베이터 여부  30000 non-null  object 
 15  주차시설_자체 여부        30000 non-null  object 
 16  주차시설_공영 여부        30000 non-null  object

In [14]:
#데이터 간략히 살펴보기
df.describe()

,개관연도,좌석수,장애인석,예매/취소구분,예매/취소매수,예매금액,예매/취소방식코드,결제수단코드,할인금액,할인종류코드,장당금액,성별,연령,난수
count,29636.000000,30000.00000,21300.000000,30000.000000,30000.0,30000.000000,29964.000000,29949.000000,30000.000000,7356.000000,30000.000000,14605.000000,16361.000000,30000.000000
mean,2004.399683,909.17760,11.156526,1.317367,1.0,47371.631667,52.889200,12.252897,2752.165533,94.631049,17256.769867,1.813831,1984.644826,0.498049
std,11.007218,745.06383,6.242002,0.465459,0.0,51128.351135,48.564698,29.920221,11871.878903,20.034269,38410.415565,0.389256,11.330803,0.289017
min,1946.000000,0.00000,1.000000,1.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1900.000000,0.000020
25%,2001.000000,272.00000,5.000000,1.000000,1.0,0.000000,2.000000,1.000000,0.000000,99.000000,0.000000,2.000000,1979.000000,0.248129
50%,2006.000000,609.00000,11.000000,1.000000,1.0,31500.000000,99.000000,2.000000,0.000000,99.000000,0.000000,2.000000,1986.000000,0.495198
75%,2012.000000,1255.00000,18.000000,2.000000,1.0,76500.000000,99.000000,2.000000,0.000000,99.000000,8000.000000,2.000000,1992.000000,0.748062
max,2019.000000,3022.00000,28.000000,2.000000,1.0,200000.000000,99.000000,99.000000,161500.000000,99.000000,200000.000000,2.000000,2020.000000,0.999998


In [16]:
#관람 수 상위 10개 공연코드 추출
df["공연코드"].value_counts().head(10)

PF377279    2855
PF379183    1679
PF378340    1608
PF375461     960
PF377721     829
PF343804     721
PF383979     645
PF371057     611
PF374862     559
PF360354     558
Name: 공연코드, dtype: int64

In [17]:
#관람 수 상위 10개 공연시설코드 추출
df["공연시설코드"].value_counts().head(10)

FC222236    3241
FC222223    3025
FC222253    2568
FC222284    1140
FC222242    1080
FC223455    1027
FC222224     990
FC223458     883
FC222055     779
FC223420     722
Name: 공연시설코드, dtype: int64

In [19]:
#관람 최다 연령 상위 20
df["연령"].value_counts().head(20)

1991.0    709
1990.0    661
1989.0    661
1992.0    659
1988.0    658
1980.0    643
1993.0    638
1987.0    613
1982.0    611
1986.0    608
1983.0    581
1985.0    572
1981.0    553
1984.0    523
1979.0    512
1994.0    488
1995.0    478
1996.0    419
1997.0    399
1977.0    387
Name: 연령, dtype: int64

In [20]:
#관람객 성별 통계
df["성별"].value_counts()

2.0    11886
1.0     2719
Name: 성별, dtype: int64

In [25]:
#관람객 예매 및 취소 통계
df['예매/취소구분'].value_counts()

1    20479
2     9521
Name: 예매/취소구분, dtype: int64

In [23]:
#관람객 성별 평균 예매금액
df.groupby(["성별"])["예매금액"].mean()

성별
1.0    70680.014711
2.0    62387.545516
Name: 예매금액, dtype: float64

In [24]:
#관람객 성별 예매금액 합
df.groupby(["성별"])["예매금액"].sum()

성별
1.0    192178960
2.0    741538366
Name: 예매금액, dtype: int64

In [41]:
#관람객 성별 평균 할인금액
df.groupby(["성별"])["할인금액"].mean()

성별
1.0    1099.337992
2.0    2280.636884
Name: 할인금액, dtype: float64

In [42]:
#관람객 성별 할인금액 합
df.groupby(["성별"])["할인금액"].sum()

성별
1.0     2989100
2.0    27107650
Name: 할인금액, dtype: int64

In [40]:
#성별, 예매 및 취소 통계
df.groupby(["성별","예매/취소구분"]).count()

전송사업자코드  전송사업자명  공연시설코드  시설특성  개관연도    주소  편의시설_레스토랑 여부  \
성별  예매/취소구분                                                            
1.0 1           1951    1951    1951  1951  1926  1951          1951   
    2            768     768     768   768   766   768           768   
2.0 1           8231    8231    8231  8231  8152  8231          8231   
    2           3655    3655    3655  3655  3626  3655          3655   

             편의시설_카페 여부  편의시설_편의점 여부  편의시설_놀이방 여부  ...  관람연령  아동공연 여부  축제 여부  \
성별  예매/취소구분                                        ...                         
1.0 1              1951         1951         1951  ...  1951     1951   1951   
    2               768          768          768  ...   768      768    768   
2.0 1              8231         8231         8231  ...  8231     8231   8231   
    2              3655         3655         3655  ...  3655     3655   3655   

             내한공연 여부  오픈런 여부  판매시작일시  판매종료일시  좌석등급  단독판매여부    난수  
성별  예매/취소구분                                                       
1.0 1           1951    1951    1951    1951  1951    1951  1951  
    2            768     768     768     768   768     768   768  
2.0 1           8231    8231    8230    8230  8231    8231  8231  
    2           3655    3655    3655    3655  3655    3655  3655  

[4 rows x 61 columns]

In [10]:
#공연코드 별 성별 관람 통계
df.groupby(["공연시설코드","성별"]).count().sort_values(by='전송사업자명', ascending=False)

,,전송사업자코드,전송사업자명,시설특성,개관연도,주소,편의시설_레스토랑 여부,편의시설_카페 여부,편의시설_편의점 여부,편의시설_놀이방 여부,편의시설_수유실 여부,...,관람연령,아동공연 여부,축제 여부,내한공연 여부,오픈런 여부,판매시작일시,판매종료일시,좌석등급,단독판매여부,난수
공연시설코드,성별,,,,,,,,,,,,,,,,,,,,,
FC222253,2.0,1311,1311,1311,1311,1311,1311,1311,1311,1311,1311,...,1311,1311,1311,1311,1311,1311,1311,1311,1311,1311
FC222236,2.0,1049,1049,1049,1049,1049,1049,1049,1049,1049,1049,...,1049,1049,1049,1049,1049,1049,1049,1049,1049,1049
FC222284,2.0,798,798,798,798,798,798,798,798,798,798,...,798,798,798,798,798,798,798,798,798,798
FC223458,2.0,657,657,657,657,657,657,657,657,657,657,...,657,657,657,657,657,657,657,657,657,657
FC222055,2.0,558,558,558,558,558,558,558,558,558,558,...,558,558,558,558,558,558,558,558,558,558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FC223424,2.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
FC223438,1.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
FC223454,1.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [4]:
df = pd.read_excel("F:/drive/KOPIS/sample_01/sample_2020_1q_01.xlsx", encoding='utf-8')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 63 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   전송사업자코드           30000 non-null  object 
 1   전송사업자명            30000 non-null  object 
 2   공연시설코드            30000 non-null  object 
 3   시설특성              30000 non-null  object 
 4   개관연도              29636 non-null  float64
 5   주소                30000 non-null  object 
 6   편의시설_레스토랑 여부      30000 non-null  object 
 7   편의시설_카페 여부        30000 non-null  object 
 8   편의시설_편의점 여부       30000 non-null  object 
 9   편의시설_놀이방 여부       30000 non-null  object 
 10  편의시설_수유실 여부       30000 non-null  object 
 11  장애인시설_주차장 여부      30000 non-null  object 
 12  장애인시설_화장실 여부      30000 non-null  object 
 13  장애인시설_경사로 여부      30000 non-null  object 
 14  장애인시설_전용엘리베이터 여부  30000 non-null  object 
 15  주차시설_자체 여부        30000 non-null  object 
 16  주차시설_공영 여부        30000 non-null  object

In [8]:
print(y_labels.shape)
print(X_features.shape)

(30000,)
(30000, 61)


In [18]:
#피처 정제
X_features_selected = df[["성별","연령","결제수단코드","좌석수"]]

In [29]:
#레이블 인코딩으로 새 데이터프레임 만들기
encoder = LabelEncoder()
gender = encoder.fit_transform(X_features_selected["성별"])
age = encoder.fit_transform(X_features_selected["연령"])
paycode = encoder.fit_transform(X_features_selected["결제수단코드"])
seatnum = encoder.fit_transform(X_features_selected["좌석수"])
y_labels_encoded = encoder.fit_transform(y_labels)
dict1 = {'gender':gender,
        'age':age,
        'paycode':paycode,
        'seatnum':seatnum}
X_features_encoded = pd.DataFrame(dict1, columns=["gender","age","paycode","seatnum"])
print(X_features_encoded)
print(y_labels_encoded)


       gender  age  paycode  seatnum
0           2   55        1      230
1           2   86        0       90
2           2   68        1       65
3           1   70        1       72
4           2   86        0      115
...       ...  ...      ...      ...
29995       1   66        1      238
29996       2   86        3      220
29997       1   51        3      129
29998       1   72        3      238
29999       2   86        0       69

[30000 rows x 4 columns]
[348  23 398 ... 719 313  22]


In [39]:
#의사결정트리 시험삼아 해보기(에러)
X_train, X_test, y_train, y_test = train_test_split(X_features_encoded,
                                                   y_labels_encoded,
                                                   test_size=0.2,
                                                   random_state=0)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
pred_proba = dt_clf.predict_proba(X_test)[:,1]
print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test,pred_proba))
print(precision_score(y_test,pred_proba, average='macro'))
print(recall_score(y_test,pred_proba, average='macro'))


0.8056666666666666
[[25  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 [ 2  0  0 ...  0  0  0]
 ...
 [ 1  0  0 ...  0  0  0]
 [10  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]]
0.0021782372260886573
0.004338394793926247


C:\Users\USER\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ValueError: multi_class must be in ('ovo', 'ovr')